<a href="https://colab.research.google.com/github/Vaibhavs10/scratchpad/blob/main/ultravox_smollm2_scratchpad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Smol-Audio

This is nothing but a YOLO run on using SmolLM2 and Whisper Turbo to test if we can do the following:

1. LLM powered transcriptions
2. Force SmolLM2 to be multilingual

Trained on HF Space, because I don't know how to use the cluster 😅

Somehow chat via Gradio didn't work, so here's a notebook with some test cases that you can play with!

In [4]:
!pip install -q datasets

In [5]:
# pip install transformers peft librosa

import transformers
import numpy as np
import librosa
import datasets

You can find the model here: https://huggingface.co/reach-vb/smollm2-whisper-turbo-uvx-14000

In [6]:
pipe = transformers.pipeline(model='reach-vb/smollm2-whisper-turbo-uvx-14000', trust_remote_code=True)

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

ultravox_config.py:   0%|          | 0.00/6.30k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/reach-vb/smollm2-whisper-turbo-uvx-14000:
- ultravox_config.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


config.json:   0%|          | 0.00/792 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Device set to use cuda:0


In [10]:
from datasets import load_dataset

cv_17 = load_dataset("mozilla-foundation/common_voice_17_0", "en", split="train", streaming=True)

print(next(iter(cv_17)))

Reading metadata...: 910420it [00:26, 21263.62it/s]'(ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: dcd8d96f-8a6e-4415-b1b7-39e81a1fbbd5)')' thrown while requesting GET https://huggingface.co/datasets/mozilla-foundation/common_voice_17_0/resolve/main/transcript/en/train.tsv
Retrying in 1s [Retry 1/5].
Reading metadata...: 1101170it [00:33, 33156.01it/s]


{'client_id': 'f15d2e0fd19c04421174108a8c02c3c2ef8e76365cdcc48090b927eca6a1d7f130bd87a104ba14cb4306adab846bb0103ba741261c64d346a94e797b9d6b659e', 'path': 'en_train_0/common_voice_en_17924809.mp3', 'audio': {'path': 'en_train_0/common_voice_en_17924809.mp3', 'array': array([ 0.00000000e+00, -2.50554802e-15, -4.35167835e-16, ...,
        3.45581235e-08,  2.53819294e-08,  2.35923494e-08]), 'sampling_rate': 48000}, 'sentence': 'Every evening, the dogs in our neighbourhood are howling.', 'up_votes': 2, 'down_votes': 0, 'age': '', 'gender': '', 'accent': '', 'locale': 'en', 'segment': '', 'variant': ''}


We use Common Voice 17 (English subset) to try and experiment stuff!

In [20]:
iter_obj = iter(cv_17)

In [44]:
sample = next(iter_obj)
audio_sample = sample["audio"]["array"]
transcript = sample["sentence"]

audio = librosa.resample(audio_sample, orig_sr=48000, target_sr=16000)

turns = [
  {
    "role": "system",
    "content": "Provide a useful answer to the audio"
  },
]

Here's the audio that's passed to the model

In [45]:
from IPython import display
display.Audio(audio, rate=16000)

and.. it's corresponding transcription

In [46]:
print(transcript)

Men that are not included in the line, the reserve, Militia Never mind, Farewell.


Here's the model response

In [47]:
pipe({'audio': audio,
      'turns': turns,
      'sampling_rate': 16000}, max_new_tokens=500)

'The phrase "Men that are not included in the line, the reserve, Militia" is quite unclear and seems to be missing some essential information. However, I can provide you with an alternative phrasing: \n\n"Men who are not part of the military line, reserve, or militia do not mind farewell."\n\nThis revised sentence conveys the same message but in a more understandable way. If you could please clarify what specific context or meaning you\'re trying to convey, I\'d be happy to help further refine your statement.'

## Test 2: Translate to German

In [48]:
sample = next(iter_obj)
audio_sample = sample["audio"]["array"]
transcript = sample["sentence"]

audio = librosa.resample(audio_sample, orig_sr=48000, target_sr=16000)

turns = [
  {
    "role": "system",
    "content": "Translate to German"
  },
]

In [49]:
from IPython import display
display.Audio(audio, rate=16000)

In [50]:
print(transcript)

Examples of the "Ziwiye Treasure" are scattered among public and private collections.


In [51]:
pipe({'audio': audio,
      'turns': turns,
      'sampling_rate': 16000}, max_new_tokens=500)

'Beispiele der Zivier-Treue sind in verschiedenen Publikationen und Privater Sammlungen verbreitet.'

# Test 3: Translate to French

In [56]:
sample = next(iter_obj)
audio_sample = sample["audio"]["array"]
transcript = sample["sentence"]

audio = librosa.resample(audio_sample, orig_sr=48000, target_sr=16000)

turns = [
  {
    "role": "system",
    "content": "Translate to French"
  },
]

In [57]:
from IPython import display
display.Audio(audio, rate=16000)

In [58]:
print(transcript)

Clemens and his friend, John Brooks, built a distillery, which helped settle the area.


In [59]:
pipe({'audio': audio,
      'turns': turns,
      'sampling_rate': 16000}, max_new_tokens=500)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


"Clément et son ami John Brooks construisent un distillier, qui a aidé à développer l'endroit."